#Installing necessary libraries

In [ ]:
!pip install transformers
!pip install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

#Models for Backtranslation

In [ ]:
# Get the name of the first model
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)

# Load the pretrained model based on the name
first_model = MarianMTModel.from_pretrained(first_model_name)

In [ ]:
# Get the name of the second model
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'

# Get the tokenizer
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained model based on the name
second_model = MarianMTModel.from_pretrained(second_model_name)

In [ ]:
import pandas as pd
squad=pd.read_csv("SQuAD.csv")
original_texts=list(squad.text)

##Eng-French Translation

In [ ]:
def format_batch_texts(language_code, batch_texts):
  
  formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]

  return formated_bach

In [ ]:
# Test of the function
format_batch_texts("fr", original_texts)

In [ ]:
def perform_translation(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

In [ ]:
squad.shape

(4693, 6)

In [ ]:
import time 
start=time.time()
translated_texts=[]
for i in range(0,4680,10):
  original_texts=list(squad['text'])[i:i+10]
  translated_texts.extend(perform_translation(original_texts, first_model, first_model_tkn))
print(f"Total Time Taken: {time.time()-start}")

##Back Translation to English

In [ ]:
start=time.time()
back_translated_texts=[]
for i in range(0,4680,10):
  back_translated_texts.extend(perform_translation(translated_texts[i:i+10], second_model, second_model_tkn))
  print(i)
print(f"Total Time Taken: {time.time()-start}")